## Part 2: Predicting market excess returns

#### Authors: Padmapriya Panneerselvam, Aditya Kothari

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed

In [2]:
Xl = pd.read_csv(r'C:\Users\ppriy\Downloads\final\final\Market TS\Xl.csv',header=None)
Xs = pd.read_csv(r'C:\Users\ppriy\Downloads\final\final\Market TS\Xs.csv',header=None)
R = pd.read_csv(r'C:\Users\ppriy\Downloads\final\final\Market TS\y.csv',header = None)

In [3]:
y_agg=R.iloc[::-1].rolling(window=12).sum().iloc[::-1]

In [4]:
t_train_start = list(range(46*12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x for x in t_test_start]

In [5]:
np.shape(t_test_end)
seed(1)
tf.compat.v1.random.set_random_seed(1234)
tf.random.uniform([1])
tf.random.normal([1])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.1468066], dtype=float32)>

In [6]:
t_test_end[551]

731

In [7]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print("retrain_idx",retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='relu'))

    model.add(BatchNormalization())

    model.add(Dense(32,activation='relu'))
    model.add(Dense(1))
    opt = tf.keras.optimizers.Adam(learning_rate=0.000001)
    model.compile(optimizer=opt,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=64, epochs=100,verbose=1,callbacks=[early_stop])
    preds.append(model.predict(X_test))
    loss.append(min(history.history['val_loss']))

retrain_idx 0
Epoch 1/100
2/2 [==============================] - 2s 382ms/step - loss: 2.2216 - val_loss: 0.0507
Epoch 2/100
2/2 [==============================] - 0s 39ms/step - loss: 2.1267 - val_loss: 0.0436
Epoch 3/100
2/2 [==============================] - 0s 34ms/step - loss: 2.0626 - val_loss: 0.0396
Epoch 4/100
2/2 [==============================] - 0s 89ms/step - loss: 2.1650 - val_loss: 0.0371
Epoch 5/100
2/2 [==============================] - 0s 37ms/step - loss: 2.5371 - val_loss: 0.0361
Epoch 6/100
2/2 [==============================] - 0s 43ms/step - loss: 2.6475 - val_loss: 0.0355
Epoch 7/100
2/2 [==============================] - 0s 51ms/step - loss: 2.2156 - val_loss: 0.0368
retrain_idx 1
Epoch 1/100
2/2 [==============================] - 2s 212ms/step - loss: 1.0311 - val_loss: 0.0333
Epoch 2/100
2/2 [==============================] - 0s 32ms/step - loss: 1.0504 - val_loss: 0.0335
retrain_idx 2
Epoch 1/100
2/2 [==============================] - 2s 242ms/step - loss: 1

In [18]:
model.summary()

Model: "sequential_551"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_normalization_551 (La  (None, 137)              274       
 yerNormalization)                                               
                                                                 
 dense_5510 (Dense)          (None, 1024)              141312    
                                                                 
 dropout_3857 (Dropout)      (None, 1024)              0         
                                                                 
 dense_5511 (Dense)          (None, 512)               524800    
                                                                 
 dropout_3858 (Dropout)      (None, 512)               0         
                                                                 
 dense_5512 (Dense)          (None, 256)               131328    
                                                    

In [12]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [13]:
y_org

,0
180,-0.277289
181,-0.145559
182,-0.080091
183,-0.034259
184,0.052814
...,...
727,0.140529
728,0.228682
729,0.171740
730,0.123710


In [14]:
preds

[array([[-0.07493076]], dtype=float32),
 array([[0.02210154]], dtype=float32),
 array([[0.0122513]], dtype=float32),
 array([[0.01052092]], dtype=float32),
 array([[-0.03056987]], dtype=float32),
 array([[0.02279887]], dtype=float32),
 array([[-0.00608103]], dtype=float32),
 array([[0.02412475]], dtype=float32),
 array([[0.0627711]], dtype=float32),
 array([[0.0217941]], dtype=float32),
 array([[0.0254554]], dtype=float32),
 array([[0.09198937]], dtype=float32),
 array([[0.04096339]], dtype=float32),
 array([[-0.02923541]], dtype=float32),
 array([[-0.02850377]], dtype=float32),
 array([[0.00333336]], dtype=float32),
 array([[0.04716238]], dtype=float32),
 array([[0.00363914]], dtype=float32),
 array([[-0.02020031]], dtype=float32),
 array([[0.03761924]], dtype=float32),
 array([[0.01339628]], dtype=float32),
 array([[0.01943045]], dtype=float32),
 array([[0.02864347]], dtype=float32),
 array([[-0.00481084]], dtype=float32),
 array([[0.01027891]], dtype=float32),
 array([[0.0472164]], 

### Calculate R^2

In [15]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.561594
dtype: float64

### A good R^2 is expected to be between 0 to 0.05.